In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [36]:
#import python libraries
import pymysql 
import pandas as pd
import numpy as np

In [37]:
zipcode = pd.read_csv('Zipcodes.csv') #using pandas to read zipcode csv
demographics_info= pd.read_csv('Demographic Data.csv') #using pandas to read demographic data csv
demographics_info = demographics_info[['JURISDICTION_ID','TOTAL POPULATION','WHITE PERCENTAGE','BLACK OR AFRICAN AMERICAN PERCENTAGE','NATIVE AMERICAN PERCENTAGE','ASIAN PERCENTAGE','PACIFIC ISLANDER PERCENTAGE','HISPANIC OR LATINO PERCENTAGE','OTHER RACE PERCENTAGE']] #selecting columns to keep
jurisdiction_dem = pd.merge(zipcode,demographics_info, left_on= 'Zipcode', right_on='JURISDICTION_ID', how='inner') #inner joining zipcode with demographic_info on zipcode
jurisdiction_dem= jurisdiction_dem.drop(['JURISDICTION_ID'], axis=1) #dropping jurisdiction_id

jurisdiction_dem['Zipcode'] =(jurisdiction_dem['Zipcode'].astype(int)) #converting data type of columns to format data types for fact_table
jurisdiction_dem['TOTAL POPULATION'] =(jurisdiction_dem['TOTAL POPULATION'].astype(int))
jurisdiction_dem['WHITE PERCENTAGE'] =(jurisdiction_dem['WHITE PERCENTAGE'].astype(float))
jurisdiction_dem['BLACK OR AFRICAN AMERICAN PERCENTAGE'] =(jurisdiction_dem['BLACK OR AFRICAN AMERICAN PERCENTAGE'].astype(float))
jurisdiction_dem['NATIVE AMERICAN PERCENTAGE'] =(jurisdiction_dem['NATIVE AMERICAN PERCENTAGE'].astype(float))
jurisdiction_dem['ASIAN PERCENTAGE'] =(jurisdiction_dem['ASIAN PERCENTAGE'].astype(float))
jurisdiction_dem['PACIFIC ISLANDER PERCENTAGE'] =(jurisdiction_dem['PACIFIC ISLANDER PERCENTAGE'].astype(float))
jurisdiction_dem['OTHER RACE PERCENTAGE'] =(jurisdiction_dem['OTHER RACE PERCENTAGE'].astype(float))

jurisdiction_demList = [tuple(l) for l in jurisdiction_dem.values.tolist()] #demographic data for jurisdiction_dim table


In [38]:
urgent_cares = pd.read_csv('Urgent Cares.csv') #using pandas to read urgent care csv
urgent_cares = urgent_cares.drop(['Phone Number', 'County'], axis=1)
urgent_cares = urgent_cares.reindex(columns=['Name','Type','Address','Zipcode','City','State'])

In [40]:
hospitals = pd.read_csv('Covid Hospitals.csv') #using pandas to read covid hospitals csv 
hospitals = hospitals.drop(['ID'], axis=1) #dropping id table
hospitals = hospitals.reindex(columns=['Name','Type','Address','Zipcode','City','State']) #rearranging order of column names to match with urgent cares/hospitals dataframe


In [41]:
frames = [urgent_cares, hospitals] #defining two data frames to merge in following lines of code
urgent_hospitals = pd.concat(frames, ignore_index=True, sort=False) #merging urgent cares with hospitals

In [42]:
minute_clinic=pd.read_csv('CVS Minute Clinic Brooklyn.csv') #using pandas to read cvs minute clinic brooklyn csv
minute_clinic = minute_clinic.drop(['Phone Number', 'County','Unnamed: 9','CVS_ID'], axis=1) #dropping columns 

In [43]:
frame1=[urgent_hospitals,minute_clinic] #defining two data frames to merge in following lines of code
testing_sites = pd.concat(frame1, ignore_index=True, sort=False) #merging urgent_hospitals with minute_clinics dataframe

In [46]:
testing_sites['Testing_ID'] = testing_sites.index + 1 #creating a testing_id column name to assign ID numbers starting from 1
testing_sites=testing_sites.dropna() #dropping all NaN values

testing_sitesList = [tuple(l) for l in testing_sites.values.tolist()] #testing_sites data for testing_dim table

In [47]:
fact_table = pd.merge(jurisdiction_dem, testing_sites, left_on= 'Zipcode', right_on='Zipcode', how='inner') #inner joining jurisdiction_dem with testing_sites on zipcode
fact_table = fact_table[['Zipcode','Testing_ID']] #selecting columns to keep
fact_table['Testing_Sites'] = 1 #creating new column Testing_Sites with row values as 1
fact_table['Zipcode'] =(fact_table['Zipcode'].astype(int)) #converting data type of columns to format data types for fact_table
fact_table['Testing_ID'] =(fact_table['Testing_ID'].astype(int))
fact_table['Testing_Sites'] =(fact_table['Testing_Sites'].astype(int))

fact_tableList = [tuple(l) for l in fact_table.values.tolist()] #fact_table data for fact_table table

In [48]:
db = pymysql.connect("127.0.0.1","root","MY_PASSWORD","coviddb") #establishing connection to MySQL Workbench

cursor = db.cursor()

In [51]:
cursor.execute("DROP TABLE IF EXISTS jurisdiction_dims;") #delete the table if it already exists

#create table with attributes based on the dimensional model
createTable1 = """CREATE TABLE jurisdiction_dims(
                 Brooklyn_Neighborhoods VARCHAR(255),
                 Zipcode int not null,
                 TOTAL_POPULATION int,
                 WHITE_PERCENTAGE FLOAT,
                 BLACK_OR_AFRICAN_AMERICAN_PERCENTAGE FLOAT,
                 NATIVE_AMERICAN_PERCENTAGE FLOAT,
                 ASIAN_PERCENTAGE FLOAT,
                 PACIFIC_ISLANDER_PERCENTAGE FLOAT,
                 HISPANIC_OR_LATINO_PERCENTAGE FLOAT,
                 OTHER_RACE_PERCENTAGE FLOAT,
                 PRIMARY KEY (Zipcode));"""

cursor.execute(createTable1)
db.commit() #commiting the create table statement

insertData1 = "INSERT INTO jurisdiction_dims VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);" #insert data into table
cursor.executemany(insertData1,jurisdiction_demList)
db.commit() 

In [52]:
cursor.execute("DROP TABLE IF EXISTS testing_dims;") #delete the table if it already exists

# create table with attributes based on the dimensional model
createTable2 = """CREATE TABLE testing_dims(
                 Name VARCHAR(255),
                 Type VARCHAR(255),
                 Address VARCHAR(255),
                 Zipcode int ,
                 City VARCHAR(255),
                 State VARCHAR(255),
                 Testing_ID int not null,
                 PRIMARY KEY (Testing_ID));"""

cursor.execute(createTable2)
db.commit() #commiting the create table statement

insertData2 = "INSERT INTO testing_dims VALUES (%s,%s,%s,%s,%s,%s,%s);" # insert data into table
cursor.executemany(insertData2,testing_sitesList)
db.commit()

In [54]:
cursor.execute("DROP TABLE IF EXISTS fact_tables;") #delete the table if it already exists

# create table with attributes based on the dimensional model
createTable3 = """CREATE TABLE fact_tables(
                  Zipcode int,
                  Testing_ID int,
                  Testing_Sites int,
                  FOREIGN KEY (Zipcode) REFERENCES jurisdiction_dims(Zipcode),
                  FOREIGN KEY (Testing_ID) REFERENCES testing_dims(Testing_ID)
                  );"""

cursor.execute(createTable3)
db.commit() #commiting the create table statement

insertData3 = "INSERT INTO fact_tables VALUES (%s,%s,%s);" #insert data into table
cursor.executemany(insertData3, fact_tableList)
db.commit()

db.close() #closing the connection to MySQL Workbench